In [1]:
import pandas as pd
import numpy as np
from numpy import asarray

from PIL import Image
from mtcnn.mtcnn import MTCNN

from numpy import load
from numpy import expand_dims
from numpy import savez_compressed

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.models import load_model

from tqdm import tqdm
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## The Main DataFrames:

In [2]:
#Load the converted metadata files for the IMDB and Wikipedia images
photo_df = pd.read_csv('photo_info_df/All_photo_data.csv')

In [3]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766


In [15]:
len(photo_df[photo_df.file_path.str.startswith('image_data/imdb_data/99/')])

3137

In [4]:
main_image_dir

CNN Basic.ipynb                       fred_image/
EDA.ipynb                             functioning functions/
FaceNet Base Model.ipynb              image_data/
FaceNet Model-Friday.ipynb            imdb_metadata_cleanup.ipynb
FaceNet Model.ipynb                   photo_info_df/
FaceNet_Model/                        photo_metadata.csv
Image To Array Notebook-Copy1.ipynb   photo_metadata_second_faces.csv
Image To Array Notebook.ipynb         wiki_metadata_cleanup.ipynb
README.md                             wiki_photo_metadata.csv
config.py                             wiki_photo_metadata_second_faces.csv
first_1000_wiki


In [4]:
#There are 511817 total images across 73399 individuals
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384199 entries, 0 to 384198
Data columns (total 8 columns):
name              384199 non-null object
dob               384199 non-null object
gender            384199 non-null float64
photo_taken       384199 non-null int64
age_when_taken    384199 non-null int64
file_path         384199 non-null object
face_location     384199 non-null object
face_score        384199 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 23.4+ MB


In [5]:
def resize_picture(filename, dimensions = (160,160), margin = 0):
            
    # load the image
    image = Image.open(filename)
    image = image.convert('RGB')
    image_array = asarray(image)
    
    #Set a margin boolean and while loop to try margin value
    margin_error = True
    
    while margin_error:
    
        try:
            #Crop the face further with MTCNN
            detector = MTCNN()
    
            #Obtain the first detected face in the cropped face picture
            first_detected_face = detector.detect_faces(image_array)[0]
        
            # get coordinates
            x1, y1, width, height = first_detected_face['box']
            x2, y2 = x1 + width + margin, y1 + height + margin       
            x1 -= margin
            y1 -= margin 
        
            face_array = image_array[y1:y2, x1:x2]
    
        except:
        
            face_array = image_array
        
        try:
            face_array_resized = Image.fromarray(face_array)
            face_array_resized = face_array_resized.resize(dimensions)
            
            margin_error = False
            break
            
        except:
            
            if margin > 0: 
                margin -= 1
            else:
                face_array_resized = Image.fromarray(image_array)
                face_array_resized = face_array_resized.resize(dimensions)
                break
    
    return asarray(face_array_resized)

In [6]:
test_df = photo_df[:100].copy()

In [7]:
tqdm.pandas()
test_df['photo_np'] = test_df['file_path'].progress_apply(lambda x: resize_picture(x, dimensions = (160,160), margin = 8))


  0%|          | 0/100 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


100%|██████████| 100/100 [02:27<00:00,  1.50s/it]


In [8]:
def plot_face(face_data):
    
    # plot face
    plt.axis('off')
    plt.imshow(face_data)

In [ ]:
plot_face(picture1)

In [10]:
x = test_df['photo_np']
y = test_df['name']

In [11]:
train, validate, test = np.split(test_df.sample(frac=1), [int(.6*len(test_df)), int(.8*len(test_df))])

In [13]:
trainx = train['photo_np']
trainy = train['name']

validatex = validate['photo_np']
validatey = validate['name']

testx = test['photo_np']
testy = test['name']


In [19]:
type(testx)

pandas.core.series.Series

In [20]:
model = Sequential()
model.add(Conv2D(64, kernel_size = 3, activation='relu', input_shape = (160, 160, 3)))
model.add(Flatten())
model.add(Dense(16, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

model.fit(trainx.values, trainy.values)

ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (60, 1)

### Preparation For The Model

In [ ]:
#The following function is from 'Deep Learning for Computer Vision' by Jason Brownlee, Page (508)

# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

### The Model

In [ ]:
# load the FaceNet model
facenet_model = load_model('FaceNet_Model/facenet_keras.h5')
facenet_model.load_weights('FaceNet_Model/facenet_keras_weights.h5')
print(facenet_model.inputs)
print(facenet_model.outputs)

In [ ]:
get_embedding(facenet_model, picture1)

In [ ]:
tqdm.pandas()
photo_df['embeddings'] = photo_df['file_path'].progress_apply(lambda x: resize_picture(x, dimensions = (160,160), margin = 8))